# Analyse and run models using FerPlus dataset

In this notebook we are going to build, test, analyze and compare the model with the previous versions. This is followed by improvements to the model and the data. We run this cycle a few times until we achieve realistic and nice results.

This model has been build in [this](https://github.com/BB8-2020/EmpathicRobot/tree/main/models/classification_model) file. 

If you have any quesentions about this notebook, you can send us a mail at maria.dukmak@student.hu.nl

In [ ]:
import sys
# You need to change this path to your project path
sys.path.append('/Users/marya/PycharmProjects/EmpathicRobot')
from conv_model import *
from models.functions import *
from models.classification_model.model_functions import *
from tensorflow.keras import Sequential

## Read data
As we have done before, our data is ready to use. In this section we will use **ferPlus** to train the model. This data has already been read, prepared and stored in [this](https://github.com/BB8-2020/EmpathicRobot/tree/main/data) file.
For now, our data is in a pickel file that we will read as follows:

For simplicity, we set up the path to the data as follows, you can also set it to your own path.

In [ ]:
os.chdir(os.getcwd() + '/data/')

We immediately split the data into train, test and validation set.

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = read_data(str('ferPlus_processed'))

As we see, the data consists of train set that contains 80% of the data. The validation and the test set are equal in size 20% and are used to subsequently test the model.

This data has already been cleaned and normalized so we don't have to do anything with the data anymore.

In [ ]:
print(f"Train set: X_train shape:{x_train.shape} Y_train shape:{y_train.shape}")

print(f"Test set: X_test shape:{x_test.shape} Y_test shape:{y_test.shape}")

print(f"Validation set: X_val shape:{x_val.shape} Y_val shape:{y_val.shape}")

## Models

In [ ]:
# We create all the models that we got 
models = build_models(input_shape=(48, 48, 1), num_classes=7)

### Model version 1 

Now it is finally time to start working on the model. We are going to start with the following model:

In [ ]:
model1 = Sequential(models[0]['layers'], name = models[0]['name'])

Let's check the summary out:

In [ ]:
model1.summary()

Looks good, time to compile!

### Compile and train

To compile the model we use Adam optimaizer and binary crossentropy as los function. Let us now train the model.

In [ ]:
compile_model(model1)

In [ ]:
history = fit_model(model1, 64, 70, False, x_train, y_train, x_val, y_val, x_test)

Now we're going to test our model using the test set for the model.

In [ ]:
test_loss, test_acc = evaluate_model(model1, x_test, y_test,  64)

In [ ]:
print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_acc:.4f}")

In [ ]:
plot_acc_loss(history)

In [ ]:
# Save the model into a tenserflow lite version
save_model_to_lite(model1, test_acc)

On the basis of the graphs we see that the model is very overfitted. Which means the results of the model are not good. We can try to solve that by making the model deeper

As we saw above, the results are not too great. Therefore we will now try to adjust the settings of the model .

### Model version 2

Now we are going the same as above. So we are going to creat the model, complie it and fit it.

In [ ]:
model2 = Sequential(models[1]['layers'], name = models[1]['name'])

In [ ]:
model2.summary()

Perfect! Lets compile 

### Compile and train

In [ ]:
compile_model(model2)

In [ ]:
history = fit_model(model2, 64, 70, False, x_train, y_train, x_val, y_val, x_test)

Now we're going to test our model using the test set for the model.

In [ ]:
test_loss, test_acc = evaluate_model(model2, x_test, y_test,  64)

In [ ]:
print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_acc:.4f}")

In [ ]:
plot_acc_loss(history)

In [ ]:
save_model_to_lite(model2, test_acc)

The results are going beter, next we are going to try to add some argumentation to the data. That could help our model to leren more. You can find the file where the data has been argumendated right [here](https://github.com/BB8-2020/EmpathicRobot/tree/main/data).

## Augment data

We split the data again:

In [ ]:
datagen, x_train_arg, y_train_arg, x_val_arg, y_val_arg, x_test_arg, y_test_arg = read_data('ferPlus_augment', True)

Now we are going to just fit the model using this data.

In [ ]:
history = fit_model(model2, 64, 100, True, datagen, x_train_arg, y_train_arg, x_val_arg, y_val_arg, x_test_arg)

Now we're going to test our model using the test set for the model.

In [ ]:
test_loss, test_acc = evaluate_model(model2, x_test_arg, y_test_arg, 64)

In [ ]:
print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_acc:.4f}")

In [ ]:
plot_acc_loss(history)

Okay! We definitely see an improvement in the model! These results are good enough for now! Let's save the model

In [ ]:
save_model_to_lite(model2, test_acc)

In [ ]:
save_all_model(model2, test_acc)

## Conclusion


As we have seen, we have saved the last model as `final model` for now. We will use this model in the final version for the project.